In [54]:
import sys
# Add utils to path
sys.path.append('../scripts')
from utils import *

In [55]:
gt = pd.read_csv('/home/tobamo/analize/project-tobamo/analysis/model/notebooks/results/merged_ranked_predictions_gold_standard.csv')
true_negatives = gt[gt['is_true_negative'] == 1]['contig_name'].tolist()
false_negatives = gt[gt['is_false_negative'] == 1]['contig_name'].tolist()
false_positives = gt[gt['is_false_positive'] == 1]['contig_name'].tolist()
true_positives = gt[gt['is_true_positive'] == 1]['contig_name'].tolist()

In [56]:
len(true_positives) + len(true_negatives) + len(false_positives) + len(false_negatives)

510

In [57]:
rf_input = pd.read_csv('/home/tobamo/analize/project-tobamo/analysis/model/results/snakemake/testing_input.csv')
orf_predictions = pd.read_csv('/home/tobamo/analize/project-tobamo/analysis/model/results/snakemake/predictions/orf_predictions.csv')
bin_df = prepare_bin_df(orf_predictions, "stacking", num_bins=10).reset_index()
contig_predictions = pd.read_csv('/home/tobamo/analize/project-tobamo/analysis/model/results/snakemake/predictions/contig_predictions.csv')

In [58]:
# Build fast lookup sets
_tp = set(true_positives)
_tn = set(true_negatives)
_fp = set(false_positives)
_fn = set(false_negatives)

# Labeling function
def _label(name: str) -> str:
    if name in _tp:
        return 'true_positive'
    if name in _tn:
        return 'true_negative'
    if name in _fp:
        return 'false_positive'
    if name in _fn:
        return 'false_negative'
    return 'unknown'

# Apply to both dataframes
rf_input['truth_label'] = rf_input['contig_name'].map(_label)
bin_df['truth_label'] = bin_df['contig_name'].map(_label)

In [59]:
# Merge orf_predictions['prediction', 'prob_1'] into rf_input by orf_name
rf_input = rf_input.merge(
    orf_predictions[['orf_name', 'prediction', 'prob_1']], 
    on='orf_name', 
    how='left'
)

print(f"rf_input shape after merge: {rf_input.shape}")
print(f"Columns added: prediction, prob_1")

rf_input shape after merge: (19421, 153)
Columns added: prediction, prob_1


In [60]:
# Merge contig_predictions['predicted_class', 'prob_1'] into bin_df by contig_name
bin_df = bin_df.merge(
    contig_predictions[['contig_name', 'predicted_class', 'prob_1']], 
    on='contig_name', 
    how='left'
)

print(f"bin_df shape after merge: {bin_df.shape}")
print(f"Columns added: predicted_class, prob_1")

bin_df shape after merge: (510, 14)
Columns added: predicted_class, prob_1


In [64]:
rf_input.to_csv('results/snakemake_rf_input_labeled.csv')
bin_df.to_csv('results/snakemake_lr_input_labeled.csv')